## Multiple Linear Regression

In [ ]:
## Run this code but do not edit it. Hit Ctrl+Enter to run the code.
# This command downloads and installs a useful package of R
# commands for constructing visualizations
install.packages("ggformula", quiet = TRUE, verbose = FALSE)

# This command loads useful packages of R commands into your
# notebook for manipulating data and constructing visualizations
suppressPackageStartupMessages({
    library(dplyr)
    library(ggplot2)
    library(ggformula)
})

# This command sets the theme for the visualizations in
# this notebook
theme_set(
  theme_light() +
  theme(
    panel.background = element_rect(fill = "white", colour = "white"),
    plot.background = element_blank(),
    axis.line = element_line(color = "black")
  )
)

#### The Dataset (`crabs.csv`)
In this notebook, we'll be using the `crabs.csv` file, which includes data about different physical attributes measured from crabs found in Boston area.

Crab is very tasty and many countries of the world import huge amount of crabs for consumption every year. The main benefits of crab farming are, labor cost is very low, production cost is comparatively lower and they grow very fast. For a commercial crab farmer knowing the right age of the crab helps them decide if and when to harvest it. Beyond a certain age, there is negligible growth in crab's physical characteristics and hence, it is important to time the harvesting to reduce cost and increase profit.

##### Variables
- `Sex` --> categorical | Male, Female, Indeterminate
- `Length` --> quantitative | Length of the Crab (in Feet)
- `Diameter` --> quantitative | Diameter of the Crab (in Feet)
- `Height` --> quantitative | Height of the Crab (in Feet)
- `Weight` --> quantitative | Weight of the Crab (in ounces)
- `Shucked.Weight` --> quantitative | Weight without the shell (in ounces)
- `Viscera.Weight` --> quantitative | Weight that wraps around your abdominal organs deep inside body (in ounces)
- `Shell.Weight` --> quantitative | Weight of the Shell (in ounces)
- `Age` --> quantitative | Age of the crab (in months)

##### Detailed description and notes can be found <a href="https://www.kaggle.com/dsv/2834512">here</a>.

### 1.0 - Exploratory Data Analysis

To begin, let's download our data. We'll download the `crabs.csv` file and store it in an R dataframe called `dat`.

In [ ]:
## Run this code but do not edit it. Hit Ctrl+Enter to run the code.
# This command saves the crabs.csv to a dataframe
url <- 'https://raw.githubusercontent.com/mahmoudharding/mas2/main/lesson15/data/crabs.csv'
dat <- read.csv(url)

**Question 1.1 -** Use the `head` command to print out the first several rows of the dataset.

In [ ]:
## Print head of dataset
# Type your response below this line


**Question 1.2 -** Use the `dim` command to find the number of crabs (rows) and number of variables (columns) in our data set.

In [ ]:
## Get dimensions of dataset
# Type your response below this line


A good measure to use to predict the age of a crab could be to use it's total weight (including the shell). When analyzing variables of interest (`Weight`, `Age`), it's often helpful to calculate summary statistics. For quantitative variables, we can use the summary command to find the five-number summary (minimum, Q1, median, Q3, maximum) and the average (mean) of the values. The code block shows how we find these summary statistics for the `Weight` variable.

**Note:** The `$` sign in R is used to isolate a single variable (`Weight`) from a dataframe (`dat`).

In [ ]:
## Run this code but do not edit it
# Find summary statistics for Weight
summary(dat$Weight)

**Question 1.3 -** Comment on what these summary statistics reveal about the `Weight` values in our dataset.

**Double-click this cell to type your answer here:**

Now let's examine `Length`. First we'll look at the summary statstics.

**Question 1.4 -** Use the `summary` command to find the 5 number summary for the variable `Length`.

In [ ]:
## Find summary statistics for Length
# Type your response below this line


**Question 1.5 -** Comment on what these summary statistics reveal about the `Length` values in our dataset.

**Double-click this cell to type your answer here:**

It is also helpful to visualize the distribution of a quantitative variable. This can be useful in determining if the values are normally distributed or skewed (right or left). First we'll take a look at the distribution of the outcome variable `Weight`.

In [ ]:
## Run this code but do not edit it
# Create histogram for Weight
gf_histogram(~Weight, data = dat, fill = 'turquoise', color = 'black')

As we suspected from the summary statistics, it appears that most of the crabs weigh less than 32 ounces. The distribution appears to have a small tail to the right, but overall it looks relatively symmetrical. Now let's take a look at the distribution of the `Length` variable.

In [ ]:
## Run this code but do not edit it
# Create histogram for Length
gf_histogram(~Length, data = dat, fill = 'turquoise', color = 'black')

**Question 1.6 -** Comment on what the histogram reveals about the `Length3` values in our dataset.

**Double-click this cell to type your answer here:**

For categorical data, it doesn't make sense to find means and medians. Instead, it's helpful to look at value counts and proportions. We can use the `table` command to find the counts of the different values for `Sex`:

In [ ]:
## Run this code but do not edit it
# Find counts of values for different sexes of crabs
sex_counts <- table(dat$Sex)
sex_counts

To get a better sense of scale, we can turn these raw counts into proportions by dividing them by the total:

In [ ]:
## Run this code but do not edit it
# Find the total by summing of all counts in table
total <- sum(sex_counts)

# Find the proportions for time by dividing table by total
sex_counts / total

**Question 1.7 -** Comment on the proportion of sex.

**Double-click this cell to type your answer here:**

Finally, to get a sense of how the weights and lengths are distributed across the various sexes we can create a box plot.

In [ ]:
## Run this code but do not edit it
# Create boxplots for weight for the different sexes
gf_boxplot(Weight ~ Sex, data = dat, fill = 'turquoise', color = 'black')

In this case, we're using `Sex` as the **predictor variable** and `Weight` as the **outcome variable**. In other words, we can use the sex of a crab (Male, Female, Indeterminate) to help predict its weight. That's because weights of different sexes of crabs are not the same. So, knowing the sex of a crab can help us better predict its weight.

**Note:** This predictor-outcome relationship is coded in R through the syntax `outcome ~ predictor`, as in `gf_boxplot(Weight ~ Sex,...)`.

We see that weight for male and female seem to have a similar distribution. The median weight for female crabs is slightly higher than male crabs. The minimum weight for the male crabs appears lower than female crabs. The crabs whose sex is indeterminate, have weights that are lower than the male and female crabs. We also notice that amongst all sexes there are a number of outliers.

### 2.0 - Scatter Plots

So, our main **outcome variable** in this analysis will be `Age`. We're going to use scatterplots to see how strongly different **predictor variables** correlate with weight. In particular, we're to explore how well each of the following variables predicts weight:

- `Weight`
- `Length`
- `Height`
- `Diameter`

To begin, let's create a **scatterplot** of `Age` and the `Length`. We can use the `gf_point` command to make the graph:

In [ ]:
## Run this code but do not edit it.
# Create scatterplot: Age ~ Weight
gf_point(Age ~ Length, data = dat)

**Question 2.1 -** Using your scatterplot, describe the relationship between `Age` and `Length`. For instance, are these variables positively or negatively related? How can you tell? Explain.

**Double-click this cell to type your answer here:**

### 3.0 - Simple Linear Regression (one predictor)

**3.1 -** Watch <a href="https://youtu.be/hvWgu4A0VA4">this video</a>, which provides an introduction to simple linear regression.

**Note:** This video is adapted from other materials and covers data from a separate context. However, the video provides a good intro to the concepts and models we'll be using in this section of the project.

Let's create a linear regression model relating `Weight` $(x)$ and `Age` $(y)$. To visualize our model, we can graph the line modeled by our equation on top of the scatterplot relating `Weight` to `Age`. We use the `gf_point` command to produce the scatterplot, the `gf_lm` command to graph our linear model, and a `%>%` symbol to put the elements together on the same graph:

In [ ]:
## Run this code but do not edit it
# Overlay linear model of Age ~ Weight on top of scatterplot
gf_point(Age ~ Weight, data = dat) %>% gf_lm(color = "red")

**Question 3.2 -** Would the slope value of the linear regression model for these data be positive or negative? How can you tell?

**Double-click this cell to type your answer here:**

R can help us find the equation that models this linear regression line. We can model a linear trend between a predictor ($x$) and outcome ($y$) using this linear regression formula:

$$
\hat{y} = \theta_{0} + \theta_{1}x
$$
Where:
- $\hat{y}$ is the predicted $y-$value (predicted outcome value)
- $\theta_{0}$ is the $y-$intercept $\rightarrow$ the predicted $y-$value (outcome value) when $x=0$ (the predictor's value is 0)
- $\theta_{1}$ is the slope $\rightarrow$ the predicted change in $y$ (outcome) for a 1-unit increase in $x$ (predictor)
- $x$ is the $x-$value (predictor value)

To fit a linear regression model to a set of data in R, we use the `lm` command. `lm` standars for "linear model." Here, we use `lm` to find the linear regression model relating `Weight` ($x$) and `Age` ($y$).

In [ ]:
## Run this code but do not edit it
# Create and display linear model: Age ~ Weight
weight_model <- lm(Age ~ Weight, data = dat)
weight_model

The output of the `lm` command is a bit clunky, but here's what it means:
- The `(Intercept)` value is the $y-$intercept ($\theta_{0}$)
- The `Weight` value is the coefficient for the predictor. In other words, it's the slope ($\theta_{1}$)

So, our regression equation can be written as:

$$
\hat{y} = 7.0104 + 0.1249x
$$

**Question 3.3 -** Identify the slope value and interpret what it means (in context).

**Double-click this cell to type your answer here:**

**Question 3.4 -** Use the `gf_point` and `gf_lm` commands to visualize a linear regression model for predicting `Age` (outcome) using `Height` (predictor).

In [ ]:
## Graph showing model for Age ~ Height
# Type your response below this line


**Question 3.5 -** Using your scatterplot, describe the relationship between `Age` and `Height`. For instance, are these variables positively or negatively related? How can you tell? Explain.

**Double-click this cell to type your answer here:**

**Question 3.6 -** Use the `lm` command to find the linear regression model you visualized above. Store the model in an object called `height_model`.

In [ ]:
## Linear model for Age ~ Height
# Type your response below this line


**Question 3.7 -** Indentify the slope value and interpret what it means (in context).

**Double-click this cell to type your answer here:**

### 4.0 - Analyzing strength $(R^2)$

In addition to the direction of a relationship (positive or negative), we can also look at the **strength** of a relationship. The strength is a measure of the **quality of our model's predictions.** A key metric for analyzing the strength of a model is $R^2$. The following diagram (from <a href = "https://skewthescript.org/3-3-a">Skew The Script</a>) shows the $R^2$ values of various linear models:

<img src="https://skewthescript.org/s/r_squared.PNG">

In the "weak" correlations, we see that our predictions (the linear model) tend to be far away from the actual data values (the points). If we used a model with weak correlation to predict **new** data values, our predictions would have high error. If we used a model with strong correlation to predict **new** data values, our predictions would have low error.

$R^2$ takes values between 0 - 1 (alternatively: $0\% - 100\%$). The stronger the model, the closer $R^2$ gets to 1 (or 100%). The weaker the model, the closer $R^2$ gets to 0 (or $0\%$). An intuitive way to think about it: for the perfectly strong correlations, the model gives 100% perfect predictions. The models explain $100\%$ of the variation in the data, so $R^2 = 100\%$. As the correlations get weaker, they start leaving room for error, since the models capture less of the variation in the data. So, the $R^2$ value declines from 100%, approaching $0\%$ if there's no correlation (model adds no prediction power compared to naive guessing).

**Optional Resource:** If you'd like a more thorough explanation of the math behind $R^2$, check out <a href="https://youtu.be/bMccdk8EdGo">this video</a>.

To see the $R^2$ values of our linear regression models, we can use the `summary` command. For example, here we get the `summary` printout of `height_model` - the modeled we stored that relates `Age` to `Height`.

In [ ]:
## Run this code but do not edit it
# Summarize Age ~ Height model
summary(height_model)

There's a lot going on in this printout. For now, focus at the bottom of the printed information. The `Multiple R-squared` value is the $R^2$ value for the model. In this case, $R^2 = 29.03\%$. So, we can say that the correlation between the age and length is relatively weak. This model would yield relatively weak predictions for age if used to predict on new crab lengths.

It's reasonable to assume that the length could be a better than height as a predictor for age. Additionally, it's may be easier to measure (even estimate for that matter) the length of a crab. Let's examine a model that uses length to predict the age.

In [ ]:
## Run this code but do not edit it
# Create and display linear model: Age ~ Length
length_model <- lm(Age ~ Length, data = dat)
length_model

This model has a slope value of 5.95. We can interpret this value to mean that for every 1 feet increase in length, we can expect about a 6 month increase in age. Since the majority of the crabs in the dataset are less than 1 foot, this particular interpretation may not be a useful.

Let's view the $R^2$ value for this model.

In [ ]:
## Run this code but do not edit it
# Summarize Age ~ Length model
summary(length_model)

What do you notice? Discuss the summary information from the `length_model` with one of your classmates.

**Question 4.1 -** Let's consider a new variable: `Diameter`. How well does the diameter of a crab predict its age? Let's start exploring. Go ahead and create a scatterplot that visualizes the relationship between `Diameter` (predictor) and `Age` (outcome). Overlay a linear regression model on the graph.

In [ ]:
## Graph showing model for Weight ~ Height
# Type your response below this line


**Question 4.2 -** Use the `lm` command to find the linear regression model you visualized above. Store the model in an object called `diameter_model`.

In [ ]:
## Linear model for Age ~ Diameter
# Type your response below this line


**Question 4.3 -** Use the `summary` command to find the $R^2$ value of your linear model.

In [ ]:
## R-squared for Age ~ Diameter
# Type your response below this line


**Question 4.4 -** When attempting to predict the age of a crab, farmers will record various measurements. Is this reasonable or is it unecessary? Justify your answers using the $R^2$ values for the `length_model` and `diameter_model`.

**Double-click this cell to type your answer here:**

### 5.0 - Motivating Multiple Regression

The scatterplots we created had rows of points with the same age for different values of `Length` and `Diamter`. Is there something going on here that we are missing?

Below is the scatterplot for `Age` and `Length`, color coded using `Diameter`.

In [ ]:
## Run this code but do not edit it
# Show scatter for Age ~ Length, color by Diameter
gf_point(Age ~ Length, color = ~Diameter, data = dat)

**Question 5.1 -** Look at the bottom-left corner of the graph. These are crabs that are not long (length lees than about 0.6 cm). Describe the diameters for these crabs.

**Double-click this cell to type your answer here:**

So far, we have only been working with simple linear regressions: models that use one predictor variable (`Length`) to predict the outcome variables (`Age`). If we'd like to use multiple predictor variables at once in order to model our outcome, we can use a technique called **multiple regression**.

For example, imagine we want to use **both** `Length` ($x_{1}$) and `Diameter` ($x_{2}$) to predict `Age` ($y$). We can write a new model with multiple predictors, like this:

$$
\hat{y} = \theta_{0} + \theta_{1}x_{1} + \theta_{2}x_{2}
$$
Where:
- $\hat{y}$ is the predicted `Age`
- $x_{1}$ is the `Length`
- $x_{2}$ is the `Diameter`

This means that...
- $\theta_{1}$ is the slope for `Length` --> the slope between `Age` and `Length`, controlling for all other predictors
- $\theta_{2}$ is the slope for `Diameter` --> the slope between `Age` and `Diameter`, controlling for all other predictors
- $\theta_{0}$ is the y-intercept --> the predicted $y-$value when $x_{1} = 0$ and $x_{2} = 0$ (when `Length` and `Diameter` are both 0)

Let's go ahead and fit this model, so we can understand what this all really means. The R code for multiple regression is quite simple. If we want to use multiple predictors within our model (such as `Length` and `Diameter`), we simply include both of them in our `lm` command. See below:

In [ ]:
## Run this code but do not edit it
# Fit multiple regression: Age ~ Length + Diameter
length_diameter_model <- lm(Age ~ Length + Diameter, data = dat)
length_diameter_model

As you can see, the values have changed a bit, and an extra slope term has now appeared in our model. We can plug these values into our model like so:

$$
\hat{y} = 2.771 + (-4.533)x_{1} + (12.859)x_{2}
$$

Here's how we can interpret the slopes in our model:
- $\theta_{1} = -4.533$ --> For every 1 cm increase in `Length`, we expect a 4.5 month **decrease** in `Age`, controlling for `Diameter`
- $\theta_{2} = 12.859$ -->  For every 1 cm increase in `Diameter`, we expect a 12.859 month **increase** in `Age`, controlling for `Length`


The key is that multiple regression allows you to **control for other predictors**, which helps us **eliminate confounding**. When we can control for diameter - i.e. when comparing crabs with similar diameter - we see that length is now **negatively** related to age. In other words, for crabs with similar diameters, we wouldn't necessarily expect the ones that are longer to be older.

So, the age of a crab is, in fact, is not positively associated with its length - as long as we're comparing crabs with similar diameters. Including diameter in our model allows us to see the underlying negative relationship between length and the age of the crab.

By contrast, we see that diameter is still positively associated with age (just not as much). The $\theta_{2}$ value (12.859) shows that, even when comparing crabs with similar lengths, the crabs with larger diameter measures will tend to be older. If we return to our graph from earlier, we can see this trend:

In [ ]:
## Run this code but do not edit it
# Show scatter for Age ~ Diameter, color by Length
gf_point(Age ~ Length, color = ~Diameter, data = dat)

Our multiple regression model found that, when controlling for diameter, age is negatively associated with length. One way to visually "control for diameter" is to only look at crabs with similar diameter measures. For example, only looking at crabs with a diamter of about 1 cm, does length appear to be negatively associated with age?

Let's explore.

First we will subset our data to find all the crabs that have a diameter measure of 1 cm.

In [ ]:
## Run this code but do not edit it
# Subset dat to only diameter measures that are equal to 1 cm
diameter_1_dat <- dat %>% subset(Diameter == 1)
head(diameter_1_dat)

**Question 5.2 -** Use the `dim` command to see how many observations we have in our subset dataframe `diameter_1_dat`.

In [ ]:
## Display the dimensions of a dataframe
# Type your response below this line


**Question 5.3 -** Use the `gf_point` command to visualize a linear regression model for predicting `Age` (outcome) using `Length` (predictor) from the `diameter_1_dat` dataframe.

In [ ]:
## Create scatterplot: Age ~ Length
# Type your response below this line



**Question 5.4 -** Only looking at crabs with a diamter of about 1 cm, does length appear to be negatively associated with age? Explain.

**Double-click this cell to type your answer here:**

**Question 5.5 -** Mention other sigifcant features that are noticable in the scatterplot. For example, are there any outliers? If so, how would they affect the slope of the linear regression equation? Explain.

**Double-click this cell to type your answer here:**

**Question 5.6 -** Go ahead and create a scatterplot that visualizes the relationship between `Length` (predictor) and `Age` (outcome) for crabs whose diameter measures are 1 cm. Overlay a linear regression model on the graph.

In [ ]:
## Run this code but do not edit it
# Overlay linear model of Age ~ Length on top of scatterplot
gf_point(Age ~ Length, data = diameter_1_dat) %>% gf_lm(color = "red")

**Question 5.7 -** Use the `lm` command to find the linear regression model you visualized above. Store the model in an object called `diameter_1_length_model`.

In [ ]:
## Linear model for Age ~ Length
# Type your response below this line


**Question 5.8 -** Use the `summary` command to find the $R^2$ value of your linear model.

In [ ]:
## R-squared for Age ~ Length
# Type your response below this line


**Question 5.9 -** What is the $R^2$ values for the `diameter_1_length_model`? Interpret the meaning of the value in context.

**Double-click this cell to type your answer here:**

###  6.0 - Model Selection

Model selection is the process of selecting one final model from among a collection of candidate models for dataset. As we have seen, fitting different models is relatively straightforward. All models have some error,therefore, the idea of a perfect or best model is not useful. Instead, we should think about a model that is "good enough".

The concepts involved in model selection are beyond the scope of this course. For now, let's stick to using R-squared as the measure for judging between models. Run the cell below to compare the R-squared values for each of our models.


In [ ]:
## Run this code but do not edit it
# Print the R-squared value for each model
print(paste("The r-squared value for the length only model is", summary(length_model)$r.squared))
print(paste("The r-squared value for the diameter only model is", summary(diameter_model)$r.squared))
print(paste("The r-squared value for the length and diameter model is", summary(length_diameter_model)$r.squared))

**6.1 -** Compare each model, then choose the model that you think is "good enough". Explain your choice based on the R-squared value and the context of what a crab farmer may or may not do on a regular basis.

**Double-click this cell to type your answer here:**